In [1]:
%load_ext autoreload
%autoreload 2

$$\begin{aligned}
&\min_X f(X) = \textrm{tr}(A^\top XB X^\top)  \\
& = \textrm{tr}(X^\top A^\top XB) & x = \textrm{vec}(X)\\
& = \left <\textrm{vec}(X),  \textrm{vec}(A^\top X B )  \right > \\
& = \left <\textrm{vec}(X), B^\top \otimes A^\top \cdot \textrm{vec}(X)  \right > \\ 
& = x^\top (B^\top \otimes A^\top) x\\ 
& = \left<AX, XB\right> \\
\mathbf{s.t.} & \\ 
&X \in \Pi_{n}
\end{aligned}$$

In [2]:
from qap_lp.models import *

In [3]:
instance_name = 'chr12a'

A0, B0 = parse(f'qapdata/{instance_name}.dat')
A = A0 / np.linalg.norm(A0)
B = B0 / np.linalg.norm(B0)
n, m = A.shape
e = np.ones(shape=(n, 1))
E = np.ones(shape=(n, n))
ab = np.kron(B.T, A.T)

param = QAPParam(A, B, n, m, e, E, ab)

## projection methods

working on problem:
### exact penalty

$$\begin{aligned}
 &\min_X f  + \mu\cdot | \textrm{tr}(XX^\top) -  n| \\
 &= f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top)  \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
&X \ge 0 \\
\end{aligned}$$

or

$$\begin{aligned}
& \min_X  f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^\top) - \Lambda \bullet X \\
\mathbf{s.t.} & \\
&Xe = X^\top e = e \\
\end{aligned}$$


very likely to become a concave function, cannot be directly solved by conic solver.



#### project point onto $D_n$

- projection may be computed by mosek
- or some other methods

#### derivatives

$$\begin{aligned}
& \nabla_X F_\mu  = A^\top XB + AXB^\top - 2\mu X \quad (-\Lambda)\\
& \nabla_\mu F_\mu  = n - \textrm{tr}(XX^\top) \\
& \nabla_\Lambda F_\mu  = - X
\end{aligned}$$

#### projected derivative

$$\begin{aligned}
&\min_D ||\nabla F_\mu + D ||_F^2  \\
\mathbf{s.t.} & \\
&D e = D^\top e = 0 \\ 
&D_{ij} \le 0 \quad \textsf{if: } X_{ij} = 1\\
&D_{ij} \ge 0 \quad \textsf{if: } X_{ij} = 0\\
\end{aligned}$$


In [387]:
nabla = QAPDerivativeL2Penalty(param)

perm = (7,5,12,2,1,3,9,11,10,6,8,4)
xol = np.zeros((n, n))
for idx, v in enumerate(perm):
    xol[idx, v-1]=1

def gt_min(x):
    return x[x >= -1e-4].min()

# initialize: x0 = [1]/n
x = x0 = np.ones((n, n)) / n
mu = 1
alpha = alpha0 = 0.1
nabla.obj(x, mu), nabla.original_obj(x)

(11.261050430793729, 0.2610504307937287)

In [ ]:
def proj_on_dn(param, x, **kwargs):
    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab
    
    model = mf.Model('projected_on_Dn')
    D = model.variable("d", [*A.shape], dom.inRange(0, 1))
    v = model.variable(1, dom.greaterThan(0.0))
    
    # m = vec(D - dF)
    m = expr.flatten(expr.sub(D, x))
    model.constraint(expr.sum(D, 0), dom.equalsTo(1))
    model.constraint(expr.sum(D, 1), dom.equalsTo(1))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol

In [ ]:
def point_proj_naive():
    # point projection
    # - stuck at an corner point, vertex.
    # - once reached, it stays.
    for i in range(5):
        obj = nabla.obj(x, mu)
        print(f'====={i }====')
        alpha = 1/(i+1)
        d0 = nabla.partial_f(x, mu)
        x0 = x - d0 * alpha0
        xp = proj_on_dn(param, x0)
    #     # direction
    #     dx = xp - x
    #     st = gt_min(x/dx)
    #     print(f"stepsize: ", alpha, st)
    #     x -=  st * alpha * dx
        x = xp
        print(f"obj: ", nabla.original_obj(x), nabla.obj(x, mu), nabla.obj(x, mu) - obj)
        print(f"stopping: ", n - x.dot(x.T).trace())
        print(x.round(3).min())
        print(np.linalg.norm(d0))

#### gradient projection

- projection may be computed by mosek
- or some other methods

In [343]:
# dF = x
def pd_on_dc(param, dF, lb_indices, ub_indices, **kwargs):

    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab

    model = mf.Model('projected_gradient_on_D_cone')
    D = model.variable("d", [*A.shape], dom.unbounded())
    v = model.variable(1, dom.greaterThan(0))
    m = expr.flatten(expr.add(D, dF))
   
    
    model.objective(mf.ObjectiveSense.Minimize, v)
    
    model.constraint(expr.sum(D, 0), dom.equalsTo(0))
    model.constraint(expr.sum(D, 1), dom.equalsTo(0))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    constrs_lb = model.constraint(D.pick(*lb_indices), dom.greaterThan(0))
    constrs_ub = model.constraint(D.pick(*ub_indices), dom.lessThan(0))

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol, model, constrs_lb, constrs_ub

In [197]:
def st(dp, x):
    model = mf.Model('step_size')
    v = model.variable(1, dom.greaterThan(0))
    V = expr.reshape(expr.repeat(v, n*n, 0), [n, n])
    delta = expr.mulElm(V, dp)
    model.constraint(expr.add(x, delta), dom.greaterThan(0))
    model.objective(mf.ObjectiveSense.Maximize, v)
#     model.setLogHandler(sys.stdout)
    try:
        model.solve()
        return v.level()[0]
    except:
        return 0

In [401]:
alpha = 0.1
# gradient projection, not good for now
for i in range(10000):
    print(f'====={i }====')
    obj = nabla.obj(x, mu)
    alpha *= 0.9
    d0 = nabla.partial_f(x, mu) 
    # lb_indices
    lb_x, lb_y = np.where(x <= 1e-5)
    # ub
    ub_x, ub_y = np.where(1-x <= 1e-5)
    dp, m, constrs_lb, constrs_ub = pd_on_dc(
        param, d0, 
        (lb_x.tolist(), lb_y.tolist()),
        (ub_x.tolist(), ub_y.tolist())
    )
    ndf = np.linalg.norm(dp)
    stp = st(dp, x)
    print(f"gradient norm: {ndf}")
    if stp <= 1e-7:
        print(f"stop @{i - 1}")
        break
    else:
        objs = [(i, nabla.obj(x - i / 10 * stp * dp, mu)) 
                for i in range(1, 11)]
        i_s, vs = min(objs, key=lambda x: x[-1])
        print(i_s, vs, stp)
        x = x + i_s / 10 * stp * dp
        print(f"obj: ", nabla.obj(x, mu), obj)
        print(f"trace deficiency: ", n - x.dot(x.T).trace())

=====0====
gradient norm: 1.4816446058693113
stop @-1


In [407]:
dp.round(4)

array([[-0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,  0.011 ,
        -0.    , -0.    , -0.3322,  0.3212, -0.    ],
       [-0.    , -0.    , -0.    , -0.    , -0.    , -0.    ,  0.1288,
        -0.    , -0.    , -0.2361,  0.1073, -0.    ],
       [-0.0888,  0.    , -0.    ,  0.1208, -0.2285,  0.    , -0.    ,
         0.0627,  0.1337, -0.    , -0.    , -0.    ],
       [-0.    , -0.1258,  0.1271, -0.    , -0.    , -0.0763, -0.    ,
        -0.    , -0.    , -0.    , -0.    ,  0.075 ],
       [ 0.0756,  0.    , -0.    , -0.    ,  0.0213, -0.2603, -0.    ,
         0.1793, -0.0159, -0.    , -0.    , -0.    ],
       [ 0.2209, -0.    , -0.    , -0.4033,  0.6215, -0.    , -0.    ,
        -0.4391, -0.    , -0.    , -0.    , -0.    ],
       [-0.    ,  0.076 ,  0.0883, -0.    , -0.    ,  0.0538, -0.    ,
        -0.    , -0.2355, -0.    , -0.    ,  0.0173],
       [-0.1639, -0.    , -0.    ,  0.1831, -0.1186, -0.    , -0.    ,
        -0.    , -0.    ,  0.0994, -0.    , -0.    ],


In [413]:
model = mf.Model('step_size')
v = model.variable(1, dom.greaterThan(0))
V = expr.reshape(expr.repeat(v, n*n, 0), [n, n])
delta = expr.mulElm(V, dp)
model.constraint(expr.add(x, delta), dom.greaterThan(0))
model.objective(mf.ObjectiveSense.Maximize, v)
model.setLogHandler(sys.stdout)
model.solve()

Problem
  Name                   : step_size       
  Objective sense        : max             
  Type                   : LO (linear optimization problem)
  Constraints            : 144             
  Cones                  : 0               
  Scalar variables       : 2               
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Optimizer terminated. Time: 0.03    


Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 0.0000000000e+00    nrm: 1e+00    Viol.  con: 0e+00    var: 

# eval

In [ ]:
x_int, _ = extract_sol_rounding(x, param.A, param.B)

In [379]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [381]:
check_obj_val(xol)

original obj 9552.0


9552.0

In [383]:
nabla.original_obj(xol), nabla.obj(xol, mu)

(0.060287558689144274, 0.060287558689143594)